# Demo of _narrative graphs_


In [1]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("rmisra/news-category-dataset", "News_Category_Dataset_v3.json")
data = pd.read_json(path, lines=True)
data.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [3]:
# create a sample
sample = data.sample(5000, random_state=42)
docs = sample["headline"] + "\n\n" + sample["short_description"]
categories = {
    "category": sample["category"],
    "region": [
        [name for name in ("u.s.", "europe", "asia", "africa", "america", "australia")
         if name in text.lower()]
        for text in docs
    ]
}
timestamps = sample["date"]

In [4]:
from narrativegraph import NarrativeGraph
from narrativegraph.nlp.extraction.spacy.naive import NaiveSpacyTripletExtractor

model = NarrativeGraph(
    triplet_extractor=NaiveSpacyTripletExtractor(
        named_entities=(2, None),
        noun_chunks=(2, None),
    ),
    sqlite_db_path="output/demo_db.sqlite",
    on_existing_db="overwrite"
) \
    .fit(docs, categories=categories, timestamps=timestamps)

INFO:narrativegraph.pipeline:Adding 5000 documents to database
INFO:narrativegraph.pipeline:Extracting triplets
Extracting triplets: 100%|██████████| 5000/5000 [00:20<00:00, 247.37it/s] 
INFO:narrativegraph.pipeline:Mapping entities and relations
INFO:narrativegraph.pipeline:Mapping triplets
Updating co-occurrence info: 100%|██████████| 11610/11610 [00:08<00:00, 1407.14it/s]


In [4]:
model.entities_

,id,label,frequency,doc_frequency,adjusted_tf_idf,first_occurrence,last_occurrence,alt_labels,category,region
0,1,new opportunities,1,1,0.000000,2014-06-20,2014-06-20,"[""new opportunities""]",[IMPACT],[]
1,2,our families,5,5,3333.333333,2012-12-22,2017-06-13,"[""our family"",""our families""]","[POLITICS, PARENTING, QUEER VOICES, IMPACT]",[]
2,3,a glorious week,1,1,0.000000,2014-02-08,2014-02-08,"[""a glorious week""]",[BUSINESS],[]
3,4,Tim Armstrong,2,1,2500.000000,2014-02-08,2014-02-08,"[""Tim Armstrong""]",[BUSINESS],[]
4,5,His company’s quarterly earnings,1,1,0.000000,2014-02-08,2014-02-08,"[""His company’s quarterly earnings""]",[BUSINESS],[]
...,...,...,...,...,...,...,...,...,...,...
14293,14294,Olympic Career,2,1,2500.000000,2016-08-14,2016-08-14,"[""Olympic Career""]",[SPORTS],[]
14294,14295,One More Gold Medal,1,1,0.000000,2016-08-14,2016-08-14,"[""One More Gold Medal""]",[SPORTS],[]
14295,14296,The winningest Olympian,1,1,0.000000,2016-08-14,2016-08-14,"[""The winningest Olympian""]",[SPORTS],[]
14296,14297,Nov 18,1,1,0.000000,2012-11-18,2012-11-18,"[""Nov 18""]",[BUSINESS],[]


In [ ]:
# create server to be viewed in own browser which blocks execution of other cells
model.serve_visualizer()

INFO:     Started server process [87834]
INFO:     Waiting for application startup.
INFO:root:Database engine provided to state before startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


In [6]:
# create server to be viewed in this notebook which therefore runs in the background
server = model.serve_visualizer(block=False)

INFO:root:Server started in background on port 8001
INFO:     Started server process [87360]
INFO:     Waiting for application startup.
INFO:root:Database engine provided to state before startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [87360]


In [7]:
server.show_iframe()

In [8]:
server.stop()

INFO:root:Background server stopped
